# Import dependencies and custom modules

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '3'  # Suppress TF log messages

import random
import numpy as np
import tensorflow as tf

In [2]:
from DatasetAPI.DataLoader import DatasetLoader
from Network.BiLSTM_with_Attention import BiLSTM_with_Attention

# Loading the dataset

In [4]:
Model = 'Attention_based_Long_Short_Term_Memory'
DIR = 'DatasetAPI\\EEG-Motor-Movement-Imagery-Dataset\\Processed-Data'
SAVE = os.path.join('Saved_Files', Model)
os.makedirs(SAVE, exist_ok=True)

# GPU memory growth configuration (TF2 style)
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

train_data_list, train_labels_list, test_data_list, test_labels_list = [], [], [], []

for folder in sorted(os.listdir(DIR)):  # Ensure correct ordering
    folder_path = os.path.join(DIR, folder)

    if os.path.isdir(folder_path):  
        
        train_data, train_labels, test_data, test_labels = DatasetLoader(DIR=folder_path)

        # Append to respective lists
        train_data_list.append(train_data)
        train_labels_list.append(train_labels)
        test_data_list.append(test_data)
        test_labels_list.append(test_labels)

train_data = np.vstack(train_data_list)
train_labels = np.vstack(train_labels_list)
test_data = np.vstack(test_data_list)
test_labels = np.vstack(test_labels_list)

# One-hot encode labels (for 4 classes)
train_labels = tf.one_hot(train_labels, depth=4)
train_labels = tf.squeeze(train_labels)
test_labels = tf.one_hot(test_labels, depth=4)
test_labels = tf.squeeze(test_labels)

In [6]:
# Model Hyper-parameters
n_input = 64       # Input size per time step
max_time = 64      # Number of time steps per sequence
lstm_size = 256    # Number of LSTM units (per direction)
attention_size = 8 # Size of the attention layer
n_class = 4        # Number of output classes
n_hidden = 64      # Hidden units in the FC layer
num_epoch = 300    # Number of training epochs
keep_rate = 0.75   # Dropout keep probability

# Learning rate parameters
initial_lr = 1e-4
lr_decay_epoch = 50   # Decay every 50 epochs
lr_decay = 0.50       # Multiply learning rate by 0.5

batch_size = 1024

In [7]:
# Create dataset objects using tf.data
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=train_data.shape[0]).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_labels))
test_dataset = test_dataset.batch(batch_size)

In [8]:
model = BiLSTM_with_Attention(max_time, n_input, lstm_size, attention_size, 1-keep_rate, n_hidden, n_class)

model.compile(optimizer=tf.keras.optimizers.Adam(initial_lr),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Learning rate scheduling and checkpointing via callbacks
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: initial_lr * (lr_decay ** (epoch // lr_decay_epoch)))
checkpoint = tf.keras.callbacks.ModelCheckpoint(os.path.join(SAVE, "BiLSTM_with_Attention.keras"), monitor='val_accuracy', save_best_only=True)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 64, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 64, 512)        │       657,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention_layer                 │ [(None, 512), (None,   │             0 │
│ (AttentionLayer)                │ 64, 1)]                │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           260 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 690,756 (2.64 MB)

 Trainable params: 690,628 (2.63 MB)

 Non-trainable params: 128 (512.00 B)

In [9]:
# Training with model.fit handles epoch and batch iterations automatically
history = model.fit(train_dataset, epochs=num_epoch, validation_data=test_dataset, callbacks=[lr_schedule, checkpoint])

Epoch 1/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 321s 4s/step - accuracy: 0.2522 - loss: 1.3926 - val_accuracy: 0.2480 - val_loss: 1.3884 - learning_rate: 1.0000e-04
Epoch 2/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 285s 4s/step - accuracy: 0.2808 - loss: 1.3822 - val_accuracy: 0.2483 - val_loss: 1.3761 - learning_rate: 1.0000e-04
Epoch 3/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 288s 4s/step - accuracy: 0.3482 - loss: 1.3225 - val_accuracy: 0.3376 - val_loss: 1.2971 - learning_rate: 1.0000e-04
Epoch 4/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 275s 3s/step - accuracy: 0.4246 - loss: 1.1527 - val_accuracy: 0.3929 - val_loss: 1.1885 - learning_rate: 1.0000e-04
Epoch 5/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 273s 3s/step - accuracy: 0.4449 - loss: 1.0888 - val_accuracy: 0.4062 - val_loss: 1.1438 - learning_rate: 1.0000e-04
Epoch 6/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 274s 3s/step - accuracy: 0.4472 - loss: 1.0679 - val_accuracy: 0.3991 - val_loss: 1.1404 - learning_rate: 1.0000e-04
Epoch 7/300
81/81 ━━━━━━━━━━━━━━━━━━━━ 275s 3s/step - accuracy: 

KeyboardInterrupt: 

In [7]:
# Evaluate the model on test data during training automatically (via validation_data)

# After training, get predictions
predictions = model.predict(test_dataset)
np.savetxt(os.path.join(SAVE, "prediction_for_test.csv"), predictions, delimiter=",")
np.savetxt(os.path.join(SAVE, "labels_for_test.csv"), test_labels.numpy(), delimiter=",")

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
